In [ ]:
import requests
import pandas as pd

# Set up of foundational data for Ivy League 
ivy_league_schools = {
    "Harvard University": {"city": "Cambridge", "state": "MA"},
    "Yale University": {"city": "New Haven", "state": "CT"},
    "Princeton University": {"city": "Princeton", "state": "NJ"},
    "Columbia University": {"city": "New York", "state": "NY"},
    "University of Pennsylvania": {"city": "Philadelphia", "state": "PA"},
    "Brown University": {"city": "Providence", "state": "RI"},
    "Dartmouth College": {"city": "Hanover", "state": "NH"},
    "Cornell University": {"city": "Ithaca", "state": "NY"}
}

base_url_schools = "https://api.data.gov/ed/collegescorecard/v1/schools"
school_dict = []

for school_name, location in ivy_league_schools.items():
    params = {
        "school.name": school_name,
        "fields": "id,school.name,school.city,school.state,location.lat,location.lon",
        "api_key": "aUYfWIfRdwDrbSfTIm3lTm5M67bWMk1YQNaqz4go"
    }
    
    response = requests.get(base_url_schools, params=params)
    data = response.json()
    
    if data.get('results'):
        for result in data['results']:
            if result.get('school.name') == school_name:
                school_dict.append(result)
                
#this was an additional data check reccomended by the apps I checked with

schools_df = pd.DataFrame(school_dict)
print("\nSchools DataFrame:")
print(schools_df)

# Step 2: Get weather data for each school
base_url_weather = "https://archive-api.open-meteo.com/v1/archive"

all_weather_data = []

for index, school in schools_df.iterrows():
    lat = school['location.lat']
    lon = school['location.lon']
    school_name = school['school.name']
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_min",
        "timezone": "America/New_York",
        "temperature_unit": "fahrenheit",
        "start_date": "2026-01-01",
        "end_date": "2026-01-31"
    }
    
    response = requests.get(base_url_weather, params=params)
    weather_data = response.json()
    
    if 'daily' in weather_data:
        # Create a row for each day
        for i, date in enumerate(weather_data['daily']['time']):
            all_weather_data.append({
                'school_name': school_name,
                'latitude': lat,
                'longitude': lon,
                'date': date,
                'temp_min': weather_data['daily']['temperature_2m_min'][i]
            })

# Create weather DataFrame
weather_df = pd.DataFrame(all_weather_data)

# Merge for the schools with weather data. Instead of merging the json files, AI recommended the data frames
combined_df = pd.merge(
    schools_df,
    weather_df,
    left_on=['school.name', 'location.lat', 'location.lon'],
    right_on=['school_name', 'latitude', 'longitude'],
    how='left'
)

# this was reccomended for cleaning the extra data up that was pulled back. 
combined_df = combined_df.drop(['school_name', 'latitude', 'longitude'], axis=1)

# Display results
print("\n" + "="*80)
print("COMBINED IVY LEAGUE SCHOOLS & WEATHER DATA")
print("="*80)
print(combined_df.head(20))

For this assignment I chose to break the locations up by Ivy League schools. I dug into some data on what regions were had the most snow and saw the North East was the most consistent. Knowing that, I decided to use Ivy Leagues since they're in the North East. 

To go with this, I also included data on the lighting in the cities. No Sun means that it'll be colder longer and that there will be more clouds. 